In [31]:
# import important library
import pandas as pd
import numpy as np
import bot

In [208]:
import importlib
importlib.reload(bot)

<module 'bot' from '/Users/henrytran/Documents/GitHub/aias_trading25/bot.py'>

# 1. Data Processing

In [2]:
# read the csv file
daily_df=pd.read_csv('BTC-Daily.csv')
daily_df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43185.48,49.006289,2.116360e+06
1,1646006400,2022-02-28 00:00:00,BTC/USD,37717.10,44256.08,37468.99,43178.98,3160.618070,1.364723e+08
2,1645920000,2022-02-27 00:00:00,BTC/USD,39146.66,39886.92,37015.74,37712.68,1701.817043,6.418008e+07
3,1645833600,2022-02-26 00:00:00,BTC/USD,39242.64,40330.99,38600.00,39146.66,912.724087,3.573010e+07
4,1645747200,2022-02-25 00:00:00,BTC/USD,38360.93,39727.97,38027.61,39231.64,2202.851827,8.642149e+07


In [3]:
# convert time strings into timestamps, and take only year, month and date values
daily_df["date"]=pd.to_datetime(daily_df["date"]).dt.strftime("%Y-%m-%d")

In [4]:
daily_df.shape

(2651, 9)

The dataset has 2651 records from 2014 to 2022.

As the data is needed for training and testing, every records before 2020 will be used for training and every records after 2020 will be used for testing.

In [5]:
# splitting the data
training = daily_df[daily_df["date"]<"2020-01-01"]
testing = daily_df[daily_df["date"]>="2020-01-01"]

In [66]:
# convert the data into csv files
training.to_csv('training.csv')
testing.to_csv('testing.csv')

After splitting, the training dataset has 1860 records and the testing data has 791 records.

In [6]:
print(training.shape)
print(testing.shape)

(1860, 9)
(791, 9)


# 2. Hill-climbing algorithm

## a. Try the bot

In [71]:
# extract the data
close_price_train=training["close"]
no_of_day_train=training.shape[0]
close_price_test =testing["close"]
no_of_day_test=testing.shape[0]

In [72]:
low_sma = bot.wma(close_price_train, 10, bot.sma_filter(10))
high_sma = bot.wma(close_price_train, 20, bot.sma_filter(20))

print(low_sma)
print(len(low_sma))
print(high_sma)
print(len(high_sma))

[-5844.114 -4371.12  -2900.949 ...   371.484   374.107   375.274]
1860
[-6466.681  -5746.248  -5014.0975 ...   354.9755   357.9115   361.117 ]
1860


In [77]:
# self-implementation for trading bot of sma
def bot_sma(close_price, hfw, lfw):
    low_sma = bot.wma(close_price, lfw, bot.sma_filter(lfw))
    high_sma = bot.wma(close_price, hfw, bot.sma_filter(hfw))
    buy_sell_signals=[]
    for i in range(len(low_sma)-1):
        if low_sma[i]<high_sma[i] and low_sma[i+1]>high_sma[i+1]:
            buy_sell_signals.append("buy")
        elif low_sma[i]>high_sma[i] and low_sma[i+1]<high_sma[i+1]:
            buy_sell_signals.append("sell")
        else:
            buy_sell_signals.append("-")
    return buy_sell_signals

## b. Hill-climbing algorithm

In [231]:
# Pseudocode for hill-climbing 

def hill_climbing(bot_type, high_frequency_window, low_frequency_window, alpha=0, max_iter=100):
    high_window = high_frequency_window
    low_window = low_frequency_window
    new_high_frequency_window = high_frequency_window
    new_low_frequency_window= low_frequency_window
    new_alpha=alpha
    cash1=0
    for i in range(max_iter):
        # find the correct parameter tweak:
        if bot_type.lower() == 'sma':
            new_high_frequency_window, new_low_frequency_window = parameter_tweak('sma', high_window, low_window)
        elif bot_type.lower() == 'smaema' and alpha !=0:
            new_high_frequency_window, new_low_frequency_window, new_alpha = parameter_tweak('smaema', high_window, low_window, alpha)
        elif bot_type.lower() == 'complex':
            new_high_frequency_window, new_low_frequency_window = parameter_tweak('complex',high_window, low_window)
            
        # run the total cash return after trading
        cash1=bot_fitness_func(bot_type, high_window, low_window, alpha)
        cash2=bot_fitness_func(bot_type, new_high_frequency_window, new_low_frequency_window, new_alpha)
        # compare the cash earned after tweaking the parameters
        if  cash2> cash1:
            high_window = new_high_frequency_window
            low_window = new_low_frequency_window
            cash1=cash2
    return high_window, low_window, cash1

def parameter_tweak(bot_type, hfw, lfw, alpha =0):
    if bot_type.lower() == 'sma':
        new_hfw, new_lfw = window_tweak(hfw, lfw)
        return new_hfw, new_lfw
    elif bot_type.lower() == 'smaema' and alpha !=0:
        new_hfw, new_lfw = window_tweak(hfw, lfw)
        new_alpha = alpha_tweak(alpha)
        return new_hfw, new_lfw, new_alpha
    elif bot_type.lower() == 'complex':
        new_hfw, new_lfw = complex_tweak(hfw, lfw)
        return new_hfw, new_lfw

def window_tweak(hfw, lfw):
    rng = np.random.default_rng()
    for _ in range(100):
        # new high frequency window
        a=rng.integers(-5,6)
        new_hfw=hfw+a
        # new low frequency window
        b=rng.integers(-5,6)
        new_lfw=lfw+b
        # we check to make sure that new_hfw in range(11,40) and new_lfw(2,10)
        new_hfw = max(2, max(2, new_hfw))
        new_lfw = max(11, min(40, new_lfw))
        if new_hfw < new_lfw:
            return new_hfw, new_lfw
    return hfw, lfw

def alpha_tweak(alpha):
    rng = np.random.default_rng()
    for _ in range(100):
        diff = rng.uniform(-0.15, 0.15)
        new_alpha = alpha + diff
        if 0.1<= new_alpha <= 1:
            return new_alpha
    return alpha # if after 100 loops and cannot find the optimal value, then we return the original alpha

def weights_tweak(weight_lst):
    rng = np.random.default_rng()
    # random index number of a weight in a list
    position=rng.integers(0,3)
    # normalised weights
    normalised_weights = [float(weight)/sum(weight_lst) for weight in weight_lst]

    # tweak the randomly selected weight with proportion different
    selected_weight_rate = normalised_weights[position]
    diff = np.random.uniform(-0.15, 0.15)
    selected_weight_rate += diff

    # the rate of other weights
    other_weight_idx= [idx for idx in range(3) if idx!=position]
    remainder_rate = 1- selected_weight_rate 
    # the remaining 2 weights, we generate randomly the proportion and multiply it with the current rates 
    one_weight_rate = rng.uniform(0.1, 0.9)
    another_weight_rate = 1- one_weight_rate
    other_weight_rates=[one_weight_rate, another_weight_rate]
    new_other_weight_rates = [remainder_rate * s for s in other_weight_rates]

    new_weights = [0,0,0]
    new_weights[position] = selected_weight_rate * sum(weight_lst)
    for i in range(2):
        idx = other_weight_idx[i]
        new_weights[idx] = new_other_weight_rates[i] * sum(weight_lst)

    return new_weights[0], new_weights[1], new_weights[2]


def complex_tweak(hfw, lfw):
    # in this code, hfw and lfw input are array-type in format of [w1, w2, w3, d1, d2, d3, sf]
    # weight tweak
    new_hfw_w1, new_hfw_w2,new_hfw_w3 = weights_tweak(hfw[0:3])
    new_lfw_w1, new_lfw_w2,new_lfw_w3 = weights_tweak(lfw[0:3])

    # window tweak
    new_hfw_d1, new_lfw_d1 = window_tweak(hfw[3], lfw[3])
    new_hfw_d2, new_lfw_d2 = window_tweak(hfw[4], lfw[4])
    new_hfw_d3, new_lfw_d3 = window_tweak(hfw[5], lfw[5])

    # alpha tweak
    new_hfw_alpha = alpha_tweak(hfw[-1])
    new_lfw_alpha = alpha_tweak(lfw[-1])

    return [new_hfw_w1, new_hfw_w2, new_hfw_w3, new_hfw_d1, new_hfw_d2, new_hfw_d3,new_hfw_alpha], [new_lfw_w1, new_lfw_w2, new_lfw_w3, new_lfw_d1, new_lfw_d2, new_lfw_d3, new_lfw_alpha]



In [243]:
#sma
hfw_result, lfw_result, cash_result = hill_climbing('sma',5,30)
hfw_result, lfw_result, cash_result

(2, np.int64(32), np.float64(1902.082955799861))

In [241]:
#smaema
hfw_result, lfw_result, cash_result = hill_climbing('smaema',5,30, 0.2)
hfw_result, lfw_result, cash_result

(np.int64(9), 40, np.float64(1367.2552154116331))

In [244]:
#complex
hfw_result, lfw_result, cash_result = hill_climbing('complex',[1,2,3,5,5,5,0.3],[3,2,1,30,30,30,0.1])
hfw_result, lfw_result, cash_result

([-0.32673453515365186,
  -0.6775756757789337,
  7.004310210932586,
  np.int64(15),
  np.int64(11),
  np.int64(18),
  0.8986501359292932],
 [1.5274477021983976,
  -0.39246832431461265,
  4.865020622116216,
  np.int64(34),
  np.int64(26),
  np.int64(31),
  0.42470924645802766],
 np.float64(112682.51986109387))

In [47]:
# final cash amount
cash_result

1000

# 3. Fitness function and Bot evaluation function

In [ ]:
def bot_fitness_func(high_window, low_window, alpha=0): # change bot_signals to high_frequency_window_size, low_frequency_window_size
    """
    This function will calculate the fitness (total cash earned from the buy/sell signals) of the trading bot
    Parameters:
    - 
    - 
    Return:
    - 
    """ 
    
    # call the bot in here
    # Jackie looks after this part - just need to call the bot with 2 parameters (high_frequency_window_size, low_frequency_window_size)
    
    # intialise bot, use training dataset for optimisation algorithms fitness functions
    bot_signals=[]
    close_price = pd.read_csv('training.csv')['close']
    
    # if bot_type.lower() == 'sma' and alpha ==0:
    bot_signals = bot.get_signals_sma2(close_price, high_window, low_window) #need to think about how to call 2 other bot algorithms
    # elif bot_type.lower() == 'smaema' and alpha!=0:
    #     bot_signals = bot.get_signals_smaema(close_price, high_window, low_window, alpha)
    # elif bot_type.lower() == 'complex' and alpha==0: #as alpha values were stored in the list of high_window and low_window
    #     bot_signals = bot.get_signals_complex(close_price, high_window, low_window)

    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    #loop through the time length
    for i in range(len(close_price)-1):
        close=close_price.iloc[i]
        # buy, ensure we have cash to buy
        if bot_signals[i] == "buy" and cash>0:
            bitcoin =  (cash*(1-fee))/close
            cash = 0
        # sell, ensure we have bitcoin to sell
        elif bot_signals[i] == "sell" and bitcoin>0:
            cash = bitcoin * close * (1-fee)
            bitcoin =0
    
    # final evaluation to change back to cash
    last_close=close_price.iloc[-1]
    if bitcoin>0:
        cash = bitcoin * last_close * (1-fee)
        bitcoin =0
    
    return cash

In [ ]:
def bot_evaluation(bot_signals, time, close_price):
    """
    This function will calculate the total cash earned from the buy/sell signals of the trading bot.
    Parameters:
    - bot_signals (list): including the buy/sell and keep signal
    - time: recorded trading time (daily)
    Return:
    - result: a nested list, with each smallest contains the time of the transaction, the amount of cash and amount of bitcoin
    """ 
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    result =[]
    
    #loop through the time length
    for i in range(len(time)-1):
        close_price=close[i]
        # buy
        if bot_signals[i] == "buy":
            bitcoin =  (cash*(1-fee))/close_price
            cash = 0
            result.append([time[i],cash, bitcoin])
        # sell
        elif bot_signals[i] == "sell":
            cash = bitcoin * close_price * (1-fee)
            bitcoin =0
            result.append([time[i],cash, bitcoin])
    
    # final evaluation to change back to cash
    close_price=close[-1]
    if bitcoin>0:
        cash = bitcoin * close_price * (1-fee)
        bitcoin =0
        result.append([time[i],cash, bitcoin])
    
    return result

In [ ]:
#def print_nicely(list):
    